#### Importimg Library

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

import os
from sklearn.manifold import TSNE
import numpy as np

from sklearn.manifold import TSNE

#### Hyperparameters

In [5]:
batch_size = 64
latent_dim = 100
lr = 1e-3
num_epochs = 2
image_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Memeriksa apakah CUDA tersedia
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("CUDA Version:", torch.version.cuda)

CUDA Available: True
GPU Name: NVIDIA GeForce GTX 1060 6GB
CUDA Version: 11.8


### Definisi Encoder & Decoder (VAE)

In [6]:
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(True),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(64 * 16 * 16, latent_dim)
        self.fc_logvar = nn.Linear(64 * 16 * 16, latent_dim)

    def forward(self, x):
        x = self.conv(x)         # (batch_size, 64*16*16)
        mu = self.fc_mu(x)       # (batch_size, latent_dim)
        logvar = self.fc_logvar(x)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(latent_dim, 64 * 16 * 16)
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 4, 2, 1),
            nn.Sigmoid()  # output [0,1]
        )

    def forward(self, z):
        x = self.fc(z).view(-1, 64, 16, 16)  # (batch_size, 64, 16, 16)
        x = self.deconv(x)                  # (batch_size, 3, 64, 64)
        return x

def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return mu + eps * std

### Dataset & DataLoader

In [7]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
])

dataset = torchvision.datasets.ImageFolder(root=r"dataset-path-here", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


### Inisialisasi Model, Optimizer, dan Loss

In [8]:
encoder = Encoder(latent_dim).to(device)
decoder = Decoder(latent_dim).to(device)

optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)

def loss_function(recon_x, x, mu, logvar):
    # recon_x = hasil rekonstruksi
    # x       = gambar asli
    # mu, logvar = parameter distribusi q(z|x)

    # Binary Cross Entropy
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    # KL Divergence
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

### Variabel untuk Logging

In [9]:
loss_history = []
lr_history = []  # untuk menyimpan learning rate di setiap iterasi

### Training Loop

In [10]:
print("Start Training VAE...")
total_iterations = 0

for epoch in range(num_epochs):
    for i, (img, _) in enumerate(dataloader):
        img = img.to(device)

        # Forward Encoder
        mu, logvar = encoder(img)
        z = reparameterize(mu, logvar)

        # Forward Decoder
        recon = decoder(z)

        # Hitung Loss
        loss = loss_function(recon, img, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Simpan loss & LR
        loss_history.append(loss.item())
        # Jika hanya satu param_group (umum di Adam), cukup ambil [0]
        lr_history.append(optimizer.param_groups[0]['lr'])

        total_iterations += 1

        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(dataloader)}], "
                  f"Loss: {loss.item():.4f}")

print("VAE Training Finished.")

Start Training VAE...
Epoch [1/2], Step [0/2290], Loss: 547730.6250
Epoch [1/2], Step [100/2290], Loss: 441571.8438
Epoch [1/2], Step [200/2290], Loss: 425328.0625
Epoch [1/2], Step [300/2290], Loss: 422743.3750
Epoch [1/2], Step [400/2290], Loss: 431723.3750
Epoch [1/2], Step [500/2290], Loss: 428694.6875
Epoch [1/2], Step [600/2290], Loss: 410803.8125
Epoch [1/2], Step [700/2290], Loss: 412631.9688
Epoch [1/2], Step [800/2290], Loss: 407981.0625
Epoch [1/2], Step [900/2290], Loss: 418553.6875
Epoch [1/2], Step [1000/2290], Loss: 413378.1875
Epoch [1/2], Step [1100/2290], Loss: 418583.0000
Epoch [1/2], Step [1200/2290], Loss: 404006.1250
Epoch [1/2], Step [1300/2290], Loss: 418499.0625
Epoch [1/2], Step [1400/2290], Loss: 410117.8438
Epoch [1/2], Step [1500/2290], Loss: 423616.7188
Epoch [1/2], Step [1600/2290], Loss: 414202.2188
Epoch [1/2], Step [1700/2290], Loss: 422017.0625
Epoch [1/2], Step [1800/2290], Loss: 416728.7188
Epoch [1/2], Step [1900/2290], Loss: 403042.0625
Epoch [1/2

### Menyimpan Model (Encoder & Decoder)

In [11]:
os.makedirs("saved_models", exist_ok=True)
encoder_save_path = "saved_models/VEE Generation 1/vae_encoder.pth"
decoder_save_path = "saved_models/VEE Generation 1/vae_decoder.pth"

torch.save(encoder.state_dict(), encoder_save_path)
torch.save(decoder.state_dict(), decoder_save_path)

print(f"Encoder disimpan ke: {encoder_save_path}")
print(f"Decoder disimpan ke: {decoder_save_path}")

Encoder disimpan ke: saved_models/VEE Generation 1/vae_encoder.pth
Decoder disimpan ke: saved_models/VEE Generation 1/vae_decoder.pth


### Visualisasi Loss & Learning Rate

In [12]:
def visualize_learning_rate_and_loss(loss_history, lr_history):
    """
    Visualisasi Training Loss dan Learning Rate selama iterasi.
    """
    plt.figure(figsize=(10, 4))
    plt.title("Training Loss (VAE)")
    plt.plot(loss_history, label="VAE Loss")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.title("Learning Rate Over Iterations")
    plt.plot(lr_history, label="Learning Rate")
    plt.xlabel("Iteration")
    plt.ylabel("LR")
    plt.legend()
    plt.show()

### Visualisasi Latent Space (t-SNE)

In [13]:
def visualize_latent_space(encoder, device, dataloader, n_samples=1000):
    """
    Mengumpulkan mu dari batch pertama yang berisi total n_samples,
    lalu melakukan t-SNE (2D) untuk melihat distribusi latent space.
    """
    encoder.eval()
    all_mu = []
    total = 0

    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            mu, logvar = encoder(images)

            # Ambil mu ke CPU, simpan
            all_mu.append(mu.cpu().numpy())

            total += images.size(0)
            if total >= n_samples:
                break

    # Gabung semua mu
    all_mu = np.concatenate(all_mu, axis=0)
    # Truncate jika melebihi n_samples
    all_mu = all_mu[:n_samples]

    # t-SNE
    print("Running t-SNE on latent vectors (mu)...")
    tsne = TSNE(n_components=2, perplexity=30, n_iter=1000)
    mu_2d = tsne.fit_transform(all_mu)

    plt.figure(figsize=(8, 6))
    plt.scatter(mu_2d[:, 0], mu_2d[:, 1], alpha=0.7, s=10, c='blue')
    plt.title("Latent Space Visualization via t-SNE")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.show()

# Contoh penggunaan:
# visualize_learning_rate_and_loss(loss_history, lr_history)
# visualize_latent_space(encoder, device, dataloader, n_samples=1000)
